In [ ]:
import sys

from FileTools import FileTools
# from pprint import pprint
from pathlib import Path
from zipfile import ZipFile
import pickle
#import random
import json
import time
import os

from matplotlib import pyplot as plt
#import seaborn as sns
import pandas as pd
import numpy as np
import math
import re

from kaggle.api.kaggle_api_extended import KaggleApi

In [ ]:
# authenticate with API

currWorkDir = os.getcwd()
userDir = Path.home()
keyPath = f"{userDir}\\PYC\\ADMIN\\kaggle.json"

with open( keyPath, 'r' ) as f: keyDict = json.load( f )
userTitle, keyTitle = keyDict.keys()
kaggleUsername, kaggleKey = keyDict[ userTitle ], keyDict[ keyTitle ]

os.environ[ 'KAGGLE_USERNAME' ] = kaggleUsername
os.environ[ 'KAGGLE_KEY' ] = kaggleKey

api = KaggleApi()
api.authenticate()


In [ ]:
# retrieve dataset
datasetOwner = 'lucafrance'
datasetName = 'the-world-factbook-by-cia'
api.dataset_download_files( f'{datasetOwner}/{datasetName}', path="." )

# await download
fTools = FileTools()
datasetFName = None
print( "Waiting for dataset download" )
while True:
    time.sleep( 1 )
    sortedFs = fTools.datesortFiles( currWorkDir, datasetName )
    if len( sortedFs ) == 0: continue
    datasetFName = list( sortedFs )[ 0 ]
    print( f"Latest: {datasetFName}" )
    break

In [ ]:
# extract and identify datafiles

origDataDir = f"{currWorkDir}\\data_or"
if not os.path.exists( origDataDir ): os.makedirs( origDataDir )

if datasetFName and Path( datasetFName ).suffix == ".zip":
    with ZipFile( datasetFName, 'r' ) as zipF: zipF.extractall( origDataDir )

dataPaths = [ f"{origDataDir}\\{pth}" for pth in os.listdir( origDataDir )
    if Path( pth ).suffix == ".csv" ]

if len( dataPaths ) > 0:
    dffBook = pd.read_csv( [ pth for pth in dataPaths ][ 0 ] )
    print( "Got dffBook DF from extracted dataset at:\n", dataPaths[ 0 ] )
else: print( "Failed get CSV" ); sys.exit()

dffBook


In [ ]:
def excludeParenth( _mtchLi, _val ):
    mtchLi = [ ]
    # # Drop value if between parentheses (pre-match open count > close count)
    for match in _mtchLi:
        matchDex = _val.index( match )
        openCount = _val[ :matchDex ].count( '(' )
        closCount = _val[ :matchDex ].count( ')' )
        parenthesised = openCount > closCount
        if not parenthesised: mtchLi.append( match )
    
    return mtchLi


def getMatchRemain( df_in, coIdex, patrn ):
    
    # get any number-pattern match from each row in a list
    mtches = df_in.iloc[ :, coIdex ].str.findall( patrn )
    # store non-number remainder of string (potential unit etc)
    rmnder = [ ]
    mtches_ret = []
    pos = 0
    for roVal in df_in.iloc[ :, coIdex ]:
        
        matchLi = mtches[ pos ]
        if type( matchLi ) != list: rmnt = None
        elif len( matchLi ) == 1: rmnt = roVal
        elif len( matchLi ) > 1: rmnt = roVal[ :roVal.index( matchLi[ 1 ] ) ]
        else: rmnt = None
        
        if rmnt:
            # is non-float and is list, exclude any parenthesised matches 
            matchLi = excludeParenth( mtches[ pos ], roVal )
            mtches_ret.append(matchLi)
            if len(matchLi)>0: 
                # exclude main value candidate  
                rmnder.append( rmnt.replace( matchLi[ 0 ], '' ) )
                
        else:
            # NEED APPEND FLOAT_ONLY MATCH; THIS FL+EMPLIST
            if type(matchLi)==float: mtches_ret.append(matchLi)  
            else: matchLi.append(np.nan)   
            rmnder.append( "" )
            
        pos += 1
    
    return mtches_ret, rmnder


    

In [ ]:
serbDex = dffBook[dffBook['Country'] == 'Serbia'].index[0]# LIMITED
serbDex

In [ ]:
# MATCHING NUMBERS
# REGEX:
#   capture group             (                 
#   zero/one                  [+-]?             possible number sign
#   1-3 nums                  \d{1,3}           up to three straight nums
#   non-capture subgroup      (?:               possible thousand-groups
#     comma and 3 nums          ,\d{3}          (sep. comma)
#     zero or more times        )*              
#   non-capture subgroup      (?:               then possible decimal part
#     decimal and 1+nums        \.\d+           
#     zero/one time             )?            
#   OR (alt. to last seq)     |                 or no groups, just              
#     0+ nums, dec, 1+nums      \d*\.\d+        more nums and poss decimal
#   OR (alt. to last seq)     |             
#     1+ nums                   \d+             or just more numbers.
#   Close capture group       )
#   ( match basic number last to capture greatest valid str segment )

patt = re.compile( r'([+-]?\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d*\.\d+|\d+)' )

dfFbDict = { }  # to collect column data during cleaning
checkTypes = { }

# colDex = 1                                        # TESTING
colDex = list(dffBook.columns).index("Environment: Total water withdrawal - agricultural")
                                                    # TESTING
for colName in dffBook.columns[ colDex: colDex+1 ]: # TESTING [ colDex: ]:
    origCol = dffBook.iloc[ :, colDex ]
    colType = dffBook[ colName ].dtype
    
    print(origCol[serbDex])                         # TESTING
    
    # get match if string, store if already float, catch unexpected
    if colType == float:
        colDict = { 'matchedNums': origCol, 'remainder': [ ] }
    elif colType == np.float64:
        colDict = { 'matchedNums': origCol.astype( float ), 'remainder': [ ] }
    else:
        matches, remainder = getMatchRemain( dffBook, colDex, patt )
        colDict = { 'matchedNums': matches, 'remainder': remainder }
        
        
        
        print(matches[serbDex])                     # TESTING
        print(remainder[serbDex])                   # TESTING
        
    
    colDict[ 'origCol' ] = origCol
    dfFbDict[ colName ] = colDict
    colDex += 1


In [ ]:
# RESTRICT TO & DISPLAY VALUES FOR SERB - AGRIWATER THROUGH PROCESS

In [ ]:
print( "This should be 660.8 million: ", 
    dffBook.loc[ dffBook[ 'Country' ] == "Serbia", 
    "Environment: Total water withdrawal - agricultural" ].iloc[ 0 ] )


In [ ]:
raise Exception("stop here")

In [ ]:
# split first match item from list as float to "clean", store else to "split"
def splitFirstOther( matchList ):
    firstVals, splitVals, checkVals = [ ], [ ], [ ]
    for mNum in range( len( matchList ) ):
        el = matchList[ mNum ]
        isFilldList = (type( el ) == list) and (len( el ) > 0)
        if isFilldList:  # remove any thousandcomma to support convert
            firstVals.append( float( ''.join( el[ 0 ].split( ',' ) ) ) )
            splitVals.append( [ v for v in el[ 1: ] ] )
        else:  # check all else are either nan or empty matchlist
            if ((type( el ) == list and len( el ) > 0) or
                (type( el ) != list and math.isnan( el ) == False)):
                checkVals.append( el )
            firstVals.append( np.nan )
            splitVals.append( np.nan )
    return firstVals, splitVals, checkVals


for colName in dfFbDict:
    colDict = dfFbDict[ colName ]
    colDict[ 'clean' ], colDict[ 'splitVals' ], colDict[ 'checkVals' ] = (
        splitFirstOther( colDict[ 'matchedNums' ] ))

# Raise message if got uncategorized data
for colName in dfFbDict:
    if len( dfFbDict[ colName ][ 'checkVals' ] ) > 0:
        print( f"Got checkvals for {colName}" )

In [ ]:
# dictionary columns to DF, checking is now float
newCols = [ ]
dfFloat = dffBook.iloc[ :, 0 ]  # start with countries
for colName in dfFbDict:
    clean = pd.Series( dfFbDict[ colName ][ 'clean' ] )
    lenFloat = len( [ i for i in clean if type( i ) == float ] )
    if lenFloat > len( clean ) * 0.90:
        newCols.append( colName )
        dfFloat = pd.concat( [ dfFloat, clean ], axis=1 )
    else: print( "col is less than 90% float. Dropping..." )

dfFloat.columns = [ 'Country' ] + newCols
dfFloat

In [ ]:
# Enforce non-nan threshold for dimensions

def nanThreshold( notNan ):  # average plus .5 standard deviation (rounded)
    return int( (sum( notNan ) / len( notNan )) + 0.5 * np.std( notNan ) )


def nonNanFromDims( dfr, dim = 1 ):
    nonNans = [ ]
    for pos in range( 0, dfr.shape[ dim ] ):
        if dim == 1: vals = dfr.iloc[ :, pos ].tolist()
        else: vals = dfr.loc[ [ pos ] ].values.tolist()[ 0 ]
        
        nonNans.append( [ vals, len( [ v for v in vals if
            type( v ) == float and not math.isnan( v ) ] ) ] )
    
    _thresh = nanThreshold( [ nval for _, nval in nonNans ] )
    keepVals = [ kval for kval, nnul in nonNans if nnul >= _thresh ]
    print( f"non-nan[ {len( keepVals )} ] thr[ {_thresh} ] dim[ {dim} ]" )
    
    return keepVals

In [ ]:
# DROP ROWS - disabled to keep all significant countries
def cleanRows():
    dfClean = pd.DataFrame( nonNanFromDims( dfFloat, dim=0 ) )
    # add a columnindex row to track names of kept columns
    dfClean.loc[ -1 ] = dfFloat.columns
    dfClean.index = dfClean.index + 1
    dfClean.sort_index( inplace=True )
    return dfClean

# dfRowsClean = cleanRows()
# keepCols_RC = nonNanFromDims( dfRowsClean, dim=1 )
# # convert to numeric df - row-cleaned
# dfRCC = pd.DataFrame( { col[ 0 ]: col[ 1: ] for col in keepCols_RC } )
# dfRCC = dfRCC.apply( pd.to_numeric, errors='ignore' )
# dfColsClean = dfRCC

In [ ]:
# CLEAN COLS to numeric, adding featname row for tracking through clean
dfFloat.loc[ -1 ] = dfFloat.columns
dfFloat.index = dfFloat.index + 1
dfFloat.sort_index( inplace=True )
keepCols = nonNanFromDims( dfFloat, dim=1 )
dfColsClean = pd.DataFrame( { col[ 0 ]: col[ 1: ] for col in keepCols } )
dfColsClean = dfColsClean.apply( pd.to_numeric, errors='ignore' )
dfColsClean.insert( 0, 'Country', dfFloat.iloc[ :, 0 ].tolist()[ 1: ] )
dfColsClean




In [ ]:
# Review clean DF
fbIsNa = dffBook.isna().sum().sum()
dfIsNa = dfColsClean.isna().sum().sum()
fbDim = dffBook.shape[ 0 ] * dffBook.shape[ 1 ]
dfDim = dfColsClean.shape[ 0 ] * dfColsClean.shape[ 1 ]
print( f"factbook originally shape: {dffBook.shape}" )
print( f"    NAN-density: {(fbIsNa / fbDim):.2f}% "
       f"({fbIsNa} NaN in {fbDim})" )
print( f"clean dataframe has shape: {dfColsClean.shape}" )
print( f"    NAN-density: {(dfIsNa / dfDim):.2f}% "
       f"({dfIsNa} NaN in {dfDim})" )

In [ ]:
# Reporting to identify any differences of unit scale within features

In [ ]:
# Add some further cleaning to remainder to reduce non-unit variations

def cleanRemainder():
    cleanPatts = [
        r'(\([^)]*\))',  # remove all bracketed
        r'(.* m)$|(.* m) '  # remove all before " m[endline/space]" (mtn names)
        ]
    
    cleanReman = { }
    
    for col in list( dfFloat.columns )[ 1: ]:
        remainder = dfFbDict[ col ][ 'remainder' ]
        for pattStr in cleanPatts:
            pattrn = re.compile( pattStr )
            remainder = remainder.str.replace( pattrn, '' )
        cleanReman.update( { colName: remainder } )
    
    return cleanReman

# cleanReman = cleanRemainder()

In [ ]:
# CHECK FOR SCALE FACTORS /GENERAL FEATURE ANALYSIS/SELECTIOn

def runScaleAnalysis( dfr, remDict ):
    colList = list( dfr.columns )
    dropFeatrs = [ ]
    cleanNotes = { }
    
    for pos in range( 1, len( colList ) ):
        colNam = colList[ pos ]
        colSeg = dfr.iloc[ :, pos ].tolist()[ :10 ]
        remndr = set( remDict[ colNam ] )
        rMainPrint = ""
        for r in list( remndr )[ :25 ]:
            if type( r ) == float: rMainPrint = rMainPrint + f"{r}\n"
            else: rMainPrint = rMainPrint + f"{r[ :60 ]}\n"
        
        report = (f"COL [ {pos} ] {colNam}\n\n"
                  f"CLEANVALS:\n{colSeg}\n\n"
                  f"REMAINDER (unq in col: {len( remndr )}):\n{rMainPrint}\n")
        
        report_a = report + "\nACCEPT(A), BREAK(B), CLEAN/SCALE NOTE(C), DROP(D)"
        report_b = report_a + "\n\nPLEASE MAKE A SELECTION:\n\n"
        usinp = input( report_a )
        while usinp not in [ 'a', 'd', 'c', 'b' ]: usinp = input( report_b )
        if usinp == 'b': break
        elif usinp == 'a': continue
        elif usinp == 'd': dropFeatrs.append( colNam )
        else: cleanNotes.update( {
            colNam: input( f"{report[ :250 ]}...\n\n\nCLEANING/SCALE NOTE" ) } )
    
    return dropFeatrs, cleanNotes

# dropFeatrs, cleanNotes = runScaleAnalysis(dfColsClean, cleanReman)

In [ ]:
# # SAVE SCALE ANALYSIS DATA
# stmp = fTools.dtStamp()
# fTools.storePKL( dropFeatrs, f'dropFeatrs_{stmp}', currWorkDir, subdir=None )
# fTools.storePKL( cleanNotes, f'cleanNotes_{stmp}', currWorkDir, subdir=None )

In [ ]:
# LOAD SCALE ANALYSIS DATA
pklFiles = [ fi for fi in
    [ open( pth, 'rb' ) for pth in
        [ list( dKey )[ 0 ] for dKey in
            [ fTools.datesortFiles( currWorkDir, fNam ) for fNam in
                [ 'dropFeatrs', 'cleanNotes' ] ] ] ] ]

dropFeats, scaleNotes = [ pickle.load( fi ) for fi in pklFiles ]
for fi in pklFiles: fi.close()

In [ ]:
df = dfColsClean.copy()
for i in dropFeats:
    try: df.drop( [ i ], axis=1, inplace=True )
    except KeyError: pass
scaleKeys = [ dkey for dkey in scaleNotes if dkey not in dropFeats ]


In [ ]:
# update value by matching remainder scale fragment via country reference

dct = dfFbDict.copy()

scaleDict = {
    "million": 1000000,
    "billion": 1000000000,
    "trillion": 1000000000000 }

cleanCountries = list( df[ 'Country' ] )

for colName in scaleKeys:
    colVals = [ ]
    row = 0
    
    # checking remnantcol (HAS PRE-CLEAN ENTRIES) for match
    for remnt in dct[ colName ][ 'remainder' ]:
        # print( remnt, "___" )
        country = dffBook[ 'Country' ][ row ]
        if country not in cleanCountries: row += 1; continue
        val = df.loc[ df[ 'Country' ] == country ][ colName ].iloc[ 0 ]
        if type( remnt ) == float: row += 1; colVals.append( val ); continue
        if remnt.startswith( "-$" ): val = 0 - val  # fix $ breaking neg float
        
        matches = [ ]
        for scale in scaleDict:  # apply lowest-index matched scale
            try: matches.append( [ remnt.index( scale ), scale ] )
            except ValueError: continue
        if len( matches ) > 0:  # sort by lowest index (first val of match)
            matchScale = sorted( matches, key=lambda x: x[ 0 ] )[ 0 ][ 1 ]
            val = val * scaleDict[ matchScale ]
        
        colVals.append( val )
        row += 1
    
    df[ colName ] = colVals

pd.options.display.float_format = '{:,.2f}'.format
df

In [ ]:
# # for colName in list(dfFbDict.keys())[5:6]:
# coldict = dfFbDict['Geography: Area - total']
# row = 0
# for rmnt in coldict['remainder']:
#     print( f"MATCH [{df['Geography: Area - total'][row]}] "
#         f"FROM RMNT: {rmnt}" )
#     row +=1

# 
# dfWater = pd.concat( [ df[ 'Country' ],
#     pd.Series( df[ 'Geography: Area - water' ] ),
#     pd.Series( df[ 'Geography: Area - total' ]) ], 
#     axis=1 ).sort_values( by=[ 'Geography: Area - water' ],
#     ascending=False )
# 
# # remnant logic change has broken scale matching
# 
# print(dfWater[ dfWater.Country == 'Chad' ])
# print(dffBook[ dffBook.Country == 'Chad' ]['Geography: Area - water'])
# print(dffBook[ dffBook.Country == 'Chad' ]['Geography: Area - total'])


In [ ]:

# raise ValueError

In [ ]:
# remove "World" until needed
dfWorld = df[ df.Country == 'World' ].copy()
df = df[ df.Country != 'World' ].copy()

In [ ]:
# a few top and bottom tens



In [ ]:
# need to go back and get units
def showTopTen( featName, _df, asc = False, subtitle = None, unit = None ):
    print( featName )
    
    df10 = pd.concat( [ _df[ 'Country' ],
        pd.Series( _df[ featName ] ) ], axis=1 ).sort_values( by=[ featName ],
        ascending=asc )[ :10 ]
    
    fig = plt.figure( facecolor="silver" )
    # plt.tight_layout()
    
    ax = fig.add_axes( [ 0, 0, 1.2, 1.2 ] )
    ax.bar( df10.iloc[ :, 0 ], df10.iloc[ :, 1 ] )
    
    title = f"{'BOTTOM' if asc else 'TOP'} TEN\n{featName}"
    if subtitle: title = f"{title}\n({subtitle})"
    if unit: title = f"{title}\n[{unit}]"
    
    ax.set_title( title, fontsize=16, ha="right", weight="demi", x=0.98,
        color="black" )
    
    ax.ticklabel_format( axis='y', useOffset=False, style='plain' )
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize( 14 )
        tick.label.set_color( 'black' )
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize( 14 )
        tick.label.set_color( 'black' )
        # tick.label.set_backgroundcolor("black")
    
    plt.xticks( rotation=45, ha='right' )
    
    plt.show()


In [ ]:
# largest countries. 
showTopTen( 'Geography: Area - total', df )
# Some thoughts:
#   The invasion of Ukraine should not be considered an act of 
#       ordinary claustrophobia
#   Antarctica: what is it good for?
#   Looking at sovereign territories as real-estate for future resources [..]
#   ...


In [ ]:
# # remnant logic change has broken scale matching

# dfWater = pd.concat( [ df[ 'Country' ],
#     pd.Series( df[ 'Geography: Area - water' ] ),
#     pd.Series( df[ 'Geography: Area - total' ]) ], 
#     axis=1 ).sort_values( by=[ 'Geography: Area - water' ],
#     ascending=False )
# 

# 
# print(dfWater[ dfWater.Country == 'Chad' ])
# print(dffBook[ dffBook.Country == 'Chad' ]['Geography: Area - water'])
# print(dffBook[ dffBook.Country == 'Chad' ]['Geography: Area - total'])
# 
# 

In [ ]:

# highest percent water area
df[ 'Water-area ratio' ] = (
    df[ 'Geography: Area - water' ] /
    df[ 'Geography: Area - total' ])
showTopTen( 'Water-area ratio', df )
# The British Indian Ocean territory data here is an outlier arising from an
#   apparent inconsistency or ambiguity in geographic description - 
#   the territory is designated across a very sparse grouping of islands which 
#   are themselves thin segments of atolls or whole/partial atolls with large 
#   inner bodies of water. Some interesting reading both for observers of 
#   colonialism and for ongoing events in the projection of sovereign power 
#   across large sea vectors:

#### British Indian Ocean Territory
![British_Indian_Ocean_Territory](https://upload.wikimedia.org/wikipedia/commons/thumb/9/96/British_Indian_Ocean_Territory_in_United_Kingdom.svg/1466px-British_Indian_Ocean_Territory_in_United_Kingdom.svg.png)

WIKI: 
>The only inhabitants are British and U.S. military personnel and associated 
contractors, who collectively number around 3,000 (2018 figures). The 
forced removal of Chagossians from the Chagos Archipelago occurred between 
1968 and 1973. [...] Today, the exiled Chagossians are still trying 
to return, saying that the forced expulsion and dispossession was unlawful, 
but the UK government has repeatedly denied them the right of return. The 
islands are off-limits to Chagossians, casual tourists, and the media. 



In [ ]:
# In the course of this study, small island groups with colonial-territorial 
# names I'm not familiar with keep showing up. Would like to dive into more 
# detail on each of these; adding "Ashmore and Cartier Islands" to the list.

ctIslands = [ 'British Indian Ocean Territory' ]

# Excluding BIOT as an outlier should reveal a more intuitive distribution.
# Better yet, to avoid catching so many islands, let's filter down to countries 
# whose coastline is no longer than their land boundaries with other countries.

showTopTen( 'Water-area ratio',
    df[ df[ 'Geography: Land boundaries - total' ] >=
        df[ 'Geography: Coastline' ] ],
    subtitle="Coastline is Equal or Smaller than Land Boundaries" )

# Just out of curiosity (good name for a blog?), the stats for completely
# landlocked countries:

showTopTen( 'Water-area ratio',
    df[ df[ 'Geography: Coastline' ] == 0 ],
    subtitle="Landlocked countries" )

In [ ]:
# It should be safe to posit that Canada is likely at least somewhat well-known 
# as a destination for those people with metal detectors you always see a few 
# hundred meters away at the beach.
showTopTen( 'Geography: Coastline', df )

In [ ]:
# countries with the largest difference between their lowest and highest point
# 'stans represent!
# while the China-Nepal border dissects Siggamartha's highest point, China 
# itself has a lower minimum elevation than Nepal.
df[ 'Maximum elevation difference' ] = (
    df[ 'Geography: Elevation - highest point' ] -
    df[ 'Geography: Elevation - lowest point' ])
showTopTen( 'Maximum elevation difference', df )

# Flattest places in the world: no point on natural ground is at an ascent of 
#   more of than five meters from any other point.
#   Cayman Islands is easiest on the hips with an M.E.D of one meter
showTopTen( 'Maximum elevation difference', df, asc=True )

# another for the ctIslands list:
ctIslands.append( 'Ashmore and Cartier Islands' )


In [ ]:
# highest percent of population in the 65+ bracket 
# Monaco and then Japan both stand conspicuously out from the distribution
# My speculation is that it would tend to be an older people who can afford 
#   Monaco's prestigious cost of living while, on the other hand, Japan has a 
#   culturally restrained birth rate - young adults tend to prioritize career 
#   ascension over family-making (cite).
showTopTen( 'People and Society: Age structure - 65 years and over', df )
# Japan takes the lead with % younger adults who might be able to 
#   take on a supporting role to the elderly, while Monaco, having more to
#   do the looking-after, is not in the bottom-ten for support ratio.
showTopTen( 'People and Society: Dependency ratios - potential support ratio',
    df, asc=True )

# another for the ctIslands list, a "remaining vestige of the once vast 
# territory of New France"; also one of the fastest-decreasing pop-rates (#4)
ctIslands.append( 'Saint Pierre and Miquelon' )

In [ ]:
# Ratio of irrigated land to total land
df[ 'Irrigated-area ratio' ] = (
    df[ 'Geography: Irrigated land' ] /
    df[ 'Geography: Area - total' ])
showTopTen( 'Irrigated-area ratio', df )


# Not surprised about most of the t10, who largely have a pastoral image,
#   but the Gaza Strip strikes one as being either desert or concrete. 
#   Still can be mostly true - Gaza as leader is still under 0.6%; an easy
#   threshold to reach given its small proportions.


def getVal( _df, _ctry, _feat ):
    return _df.loc[ _df[ 'Country' ] == _ctry, _feat ].iloc[ 0 ]


print( f"Irrigated area in Gaza Strip is "
       f"{getVal( df, 'Gaza Strip', 'Irrigated-area ratio' )} km" )

#   Some reading on esyrt in Palestinian agriculture: 
#       https://socialsciences.mcmaster.ca/kubursi/ebooks/water.htm

In [ ]:
# Syria's ascendency in population growth must be linked to its even 
# greater leadership in net migration. What's the story there?
# (once the Cocos is excluded as an outlier. Also what's the story? 
#   In other sources the pop has not seemingly changed by anything like 100%)
#  Monaco receiving a steady stream of aspiring casino magnates

showTopTen( 'People and Society: Population growth rate', df )
showTopTen( 'People and Society: Net migration rate', df )

# Most emmigration, and populations in greatest contraction
# two very strong categories: either islands (esp. Pacific), or eastern Europe.
# (Jordan being the one exception)
showTopTen( 'People and Society: Net migration rate', df, asc=True )
showTopTen( 'People and Society: Population growth rate', df, asc=True )

# more for the ctIslands list
ctIslands.append( 'Cocos (Keeling) Islands' )
ctIslands.append( 'Anguilla' )
ctIslands.append( 'British Virgin Islands' )
ctIslands.append( 'Cayman Islands' )  # may as well get into these while we're here
ctIslands.append( 'Northern Mariana Islands' )



In [ ]:
# Qatar & UAE's men-to-women ratios in the 25-54yo, 55-64yo and 65+ brackets 
# are startlingly weighted to men, the strongest in the world.
showTopTen( 'People and Society: Sex ratio - 25-54 years', df )
showTopTen( 'People and Society: Sex ratio - 55-64 years', df )
# At the same time, both have (by good measure) the highest value for
# the percentage of total population that is 25-54 years old.
showTopTen( 'People and Society: Age structure - 25-54 years', df )
# Interesting combination - purely speculating, one can imagine a dominating
# social discourse might concern the relation of older men to working-age adults.
# This consideration is furnished with the dependency ratio, in which
# UAE and Qatar are by very, very far the the world chart-toppers:
showTopTen( 'People and Society: Dependency ratios - potential support ratio', df )
# (What is the DR, and how would these three figures relate?)

# meanwhile, in Palau and North Korea, in the 65+ bracket, there are more than 
# twice as many old codgettes as old codgers.
showTopTen( 'People and Society: Sex ratio - 65 years and over', df, asc=True )

# for for the Kooky Islands Krew
ctIslands.append( 'Saint Barthelemy' )
ctIslands.append( 'Faroe Islands' )
ctIslands.append( 'Turks and Caicos Islands' )

In [ ]:
# People and Society: Current Health Expenditure
# Surprised to see US near the top with the impression given by private health 
# costs.
showTopTen( 'People and Society: Current Health Expenditure', df )
# Only one country in t10 expenditure is also in t10 physician density (Cuba)
showTopTen( 'People and Society: Physicians density', df )
# physician density strikes me as vital, as a population's relationship with
#   health-seeking, at a preventative stage, [is tied] to its familiarity and 
#   access to responsive human consultation.
# Nice littel one-liner aside: Cuba provides more medical personnel to the 
#   developing world than all the G8 countries combined.

# The only "first-world" country in bottom-10 health expenditure, indeed
#   leading it, is Monaco, which happens also to:
#       - have the second-highest physician density
#       - top the "% pop is 65+"
#   So, with such an old population and such a high density of physicians, 
#   it may be safe to assume that it is an excellent destination for 
#   private medical enterprise
showTopTen( 'People and Society: Current Health Expenditure', df, asc=True )

# for for the Kooky Islands Krew
ctIslands.append( 'San Marino' )

In [ ]:
# Southern Africa nations exclusively form the t10 %pop with HIV-AIDs
df[ 'People living with HIV/AIDs as percentage of population' ] = (
    df[ 'People and Society: HIV/AIDS - people living with HIV/AIDS' ] /
    df[ 'People and Society: Population' ])
showTopTen( 'People living with HIV/AIDs as percentage of population', df )

# one becomes curious which SA nation has the lowest population with HIV/AIDs
sthEquatAfrica = dffBook[
    (dffBook[ "Geography: Map references" ] == "Africa")
    & (dffBook[ "Geography: Geographic coordinates" ].str.contains( "S" ))
].Country

showTopTen( 'People living with HIV/AIDs as percentage of population',
    df[ df[ 'Country' ].isin( sthEquatAfrica ) ], asc=True,
    subtitle="African Nations South of Equator" )

For the countries that are not islands, Angola - being large, and close to the
HIV/AIDs epicentre, appears to have some form of strongly inhibiting factor.
A look at recent history identifies a cause for the low prevalence: civil war. 

> The 27-year civil war in Angola, lasting from 1975 until 2002, kept the spread 
> of HIV to a minimum due to large parts of the country being inaccessible to 
> people infected with the virus. During the civil war, individuals from 
> neighboring countries such as Zambia, Botswana, and Zimbabwe (all countries 
> with high prevalence rates of HIV) were also not allowed to come into the 
> country, which played a significant role in controlling the spread of HIV.
> [(source: Wikipedia)](https://en.wikipedia.org/wiki/HIV/AIDS_in_Angola#History)



In [ ]:
# more for the Kooky Islands Krew
ctIslands.append( 'Comoros' )
ctIslands.append( 'Sao Tome and Principe' )

In [ ]:
# Nearly half the people in Nauru and Burma smoke
showTopTen( 'People and Society: Tobacco use - total', df )
# However, when limited to females, whereas European nations remain in the t10,
# the Asia-Pacific nations Burma, Kiribati, Timor Leste, PNG and Indonesia 
# disappear ( whereas for men they remain).  
#   - Gender associations with the act of smoking? 
#     (eg.: "smoking is manly (positive) and/or unwomanly (negative)"

# Pattern strikingly reversed for Nauru (#1 t10:total/women, not in t10:men).
showTopTen( 'People and Society: Tobacco use - male', df )
showTopTen( 'People and Society: Tobacco use - female', df )

# No Smoking:
showTopTen( 'People and Society: Tobacco use - total', df, asc=True )


In [ ]:
# who has the highest combined score for both adult obesity prevalence and 
# children 4- underweight, where both scores are above feature mean?

uFeat = 'People and Society: Children under the age of 5 years underweight'
oFeat = 'People and Society: Obesity - adult prevalence rate'

df[ 'Generational weight disparity' ] = (df[ uFeat ] + df[ oFeat ])

aboveMeans = [ country for country in df[ 'Country' ] if (
    df.loc[ df[ 'Country' ] == country, uFeat ].iloc[ 0 ]
    >= df[ uFeat ].mean() and
    df.loc[ df[ 'Country' ] == country, oFeat ].iloc[ 0 ]
    >= df[ oFeat ].mean()) ]

showTopTen( 'Generational weight disparity',
    df[ df[ 'Country' ].isin( aboveMeans ) ],
    subtitle="Adult obesity and children underweight both above-mean" )

In [ ]:
# EDUCATION DISPARITIES:
expectFeatStr = (
    "People and Society: School life expectancy (primary to tertiary "
    "education)")
showTopTen( f'{expectFeatStr} - total', df )
showTopTen( f'{expectFeatStr} - male', df )
showTopTen( f'{expectFeatStr} - female', df )

# Pat on the back for Aus: by a modest yet significant margin, has the longest
#   total percent competing teriary, AND the position holds true for women as 
#   much as men.

In [ ]:
def getRank( _df, ctry, feature ):
    value = _df[ feature ].loc[ _df[ 'Country' ] == ctry ].values[ 0 ]
    if str(value) == 'nan': return print( f"{ctry} is null for\n{feature}" )
    rank = len( [ v for v in pd.Series( _df[ feature ] ) if v < value ] )
    ties = len( [ v for v in pd.Series( _df[ feature ] ) if v == value ] ) - 1
    print( f"With value of [ {value} ], {ctry} is {rank}th-highest for:\n"
           f"'{feature}'\n(out of total {len( cleanCountries )} ranked)" )
    if ties > 0: print( f"TIED WITH {ties} COUNTRIES" )

In [ ]:

# A pleasant pit-stop in the forests of Suriname:
showTopTen( 'Environment: Land use - forest', df )

# interestingly, Suriname does very little to leverage this as a resource 
#   advantage - Suriname's rank in %rev Forst rsrc is 174th.

getRank( df, 'Suriname',
    'Environment: Revenue from forest resources - forest revenues' )


In [ ]:
#   Fairly even, City-States understandably dominating the top 10. 
showTopTen( 'Environment: Urbanization - urban population', df )
showTopTen( 'Environment: Urbanization - urban population', df, asc=True )

# Of the least-urbanised, several are pacific territories with relatively 
#   unfamiliar names: Wallis and Futuna (FR), Montserrat (UK) and Tokelau (NZ). 
#   To help with attribution, if not decolonization, I submit that the UK and 
#   French territories swap names.

# Liechtenstein... is also there.
# "It is a testimony to the mere political expediency of the purchase that the Princes of Liechtenstein did not visit 
#   their new principality for almost 100 years." [citation needed]

# for the Kooky Islands Krew
ctIslands.append( 'Wallis and Futuna' )
ctIslands.append( 'Montserrat' )
ctIslands.append( 'Tokelau' )


### COAL

#### Not a glitch: China's appetite

In [ ]:
showTopTen( 'Energy: Coal - Production', df )
showTopTen( 'Energy: Coal - Consumption', df )

Just for scale, this is what the production of the top-ten coal producers 
looks like. More astonishingly, China remains a net importer - they consume 
this and more. The equivalent plot for consumption looks the same, except eg.
Australia disappears down to 200th in the world; quite a feat for the fourth-
largest producer, whereas the rest of the top ten producers are in the top ten 
consumers (excepting Kazakhstan, who drops out similarly as consumer to 198th).

In [ ]:
# update this to return a percap or perGDP ranking.
getRank( df, 'Australia', 'Energy: Coal - Consumption' )
getRank( df, 'Kazakhstan', 'Energy: Coal - Consumption' )


Some figures in greater detail:

PRODUCTION: China's production is [ 1.09 ] times that of the rest of the 
world <br>
>   [ 4,314,681,000.00 ]: China's coal production<br>
>   [ 3,974,250,000.00 ]: rest of world combined<br>
 
CONSUMPTION: China's consumption is [ 1.27 ] times that of the rest of the 
world<br>
>   [ 4,506,387,000.00 ]: China's coal consumption<br>
>   [ 3,534,985,000.00 ]: rest of world combined<br>
   

In [ ]:
prodFeat = 'Energy: Coal - Production'
consFeat = 'Energy: Coal - Consumption'

chinaProd = df[ df.Country == 'China' ][ prodFeat ].sum()
chinaCsmp = df[ df.Country == 'China' ][ consFeat ].sum()

notChinaProd = df[ df.Country != 'China' ][ prodFeat ].sum()
notChinaCsmp = df[ df.Country != 'China' ][ consFeat ].sum()

# get longest string length to pad report field
pad = (max( [ len( str( i ) )
    for i in [ chinaProd, notChinaProd, chinaCsmp, notChinaCsmp ] ] ))

productReport = (f"PRODUCTION: China's production is "
                 f"[ {(chinaProd / notChinaProd):,.2f} ] times that of the rest of world\n"
                 f"   [ {chinaProd:>{pad},.2f} ]: China's coal production\n"
                 f"   [ {notChinaProd:>{pad},.2f} ]: rest of world combined\n")
consmptReport = (f"\nCONSUMPTION: China's consumption is "
                 f"[ {(chinaCsmp / notChinaCsmp):,.2f} ] times the size\n"
                 f"of the rest of world combined.\n"
                 f"   [ {chinaCsmp:>{pad},.2f} ]: China's coal consumption\n"
                 f"   [ {notChinaCsmp:>{pad},.2f} ]: rest of world combined\n")

print( productReport, consmptReport )

In [ ]:
# Corobborating the disparity between production and consumption, the export
#   figures appear as follows.
showTopTen( 'Energy: Coal - Exports', df )

# We can get an image of a country's relationship with coal if we look at the 
#   combined production and imports in ratio to exports. Where this ratio is 
#   above one, a country has exported abve the total it produced and imported,
#   meaning it has sold reserves.
#   Refining this further to only the countries whose exports are above the
#   world-mean, we can see who has a strong reliance on coal exports.

df[ 'Coal: Holdings-to-Export ratio' ] = (
    df[ 'Energy: Coal - Exports' ] /
    (df[ 'Energy: Coal - Production' ] +
     df[ 'Energy: Coal - Imports' ]))

aboveMeans = [ country for country in df[ 'Country' ] if (
    df.loc[ df[ 'Country' ] == country, 'Energy: Coal - Exports' ].iloc[ 0 ]
    >= df[ 'Energy: Coal - Exports' ].mean()) ]

showTopTen( 'Coal: Holdings-to-Export ratio',
    df[ df[ 'Country' ].isin( aboveMeans ) ] )

# In order, the refinement exluded Venezuela, Belarus and Eswatini
# such that Russia, South Africa and the Phillipines entered the t10 "sellers"


# Environment: Revenue from coal - coal revenues
#   "% revenue derived from taxation on coal production"?
#   Compare with "Exports" [...]
showTopTen( 'Environment: Revenue from coal - coal revenues', df )


# Naturally, on the topic of coal, a look at the top CO2 emitters:

showTopTen( 'Environment: Air pollutants - carbon dioxide emissions', df )
#   China leads by twice its nearest competitor, the US. See coal-relation (hah)
# Similar stats for Environment: Air pollutants - methane emission.


In [ ]:
# Who are the most and least trade-reliant for water? (Hi there, Middle East)
# Withdrawrals (municipal, industrial and agricultural) minus renewable sources
df[ 'Water withdrawal exposure to trade' ] = (
    (df[ 'Environment: Total water withdrawal - municipal' ] +
     df[ 'Environment: Total water withdrawal - industrial' ] +
     df[ 'Environment: Total water withdrawal - agricultural' ]) -
    df[ 'Environment: Total renewable water resources' ])

showTopTen( 'Water withdrawal exposure to trade', df, 
    subtitle="Withdrawrals minus resources" )

# Brazil is sitting happy there around the Amazon. Russia and Canada just 
# melt vast amounts of snow.
showTopTen( 'Water withdrawal exposure to trade', df, asc=True, 
    subtitle="Withdrawrals minus resources" )

In [ ]:
# Somehow, Serbia's 'Environment: Total water withdrawal - agricultural'
#    ORIG:   660.8 million cubic meters (2017 est.)
# BECOMES:   900,000,000,000.00

dfWater = pd.concat( [ 
    df[ 'Country' ],
    df[ 'Environment: Total water withdrawal - municipal' ],
    df[ 'Environment: Total water withdrawal - industrial' ],
    df[ 'Environment: Total water withdrawal - agricultural' ],
    df[ 'Environment: Total renewable water resources' ] ], 
    axis=1 ).sort_values( 
        by=[ 'Environment: Total water withdrawal - agricultural' ], 
        ascending=False )[ :10 ]

# dfWater[ dfWater[ 'Country' ] == "Serbia" ]
# dfWaterFB[ dfWaterFB[ 'Country' ] == "Serbia" ]
print( "This should be 660.8 million: ", 
    dfWater.loc[ dfWater[ 'Country' ] == "Serbia", 
    "Environment: Total water withdrawal - agricultural" ].iloc[ 0 ] )
    

In [ ]:
dfWaterFB = pd.concat( [ 
    dffBook[ 'Country' ],
    dffBook[ 'Environment: Total water withdrawal - municipal' ],
    dffBook[ 'Environment: Total water withdrawal - industrial' ],
    dffBook[ 'Environment: Total water withdrawal - agricultural' ],
    dffBook[ 'Environment: Total renewable water resources' ] ], 
    axis=1 )

# dfWaterFB[ dfWaterFB[ 'Country' ] == "Serbia" ]
print( "This should be 660.8 million: ", 
    dfWaterFB.loc[ dfWaterFB[ 'Country' ] == "Serbia", 
    "Environment: Total water withdrawal - agricultural" ].iloc[ 0 ] )

In [ ]:
ctIslands.append('Cape Verde')
ctIslands.append('Saint Vincent and the Grenadines')
ctIslands.append('Saint Kitts and Nevis')
ctIslands.append('Saint Lucia')

In [ ]:
'Economy: Real GDP (purchasing power parity)'
# Something funky going on with Iran 

# It's taken a "trillion" relevant to a remnant-value (2018) and applied it
#   to the firstmatch-value (2020). 

# if second match, truncate remainder at 

# Need to truncate all remainder at position of second match.
#     

# IRAN REAL GDP
#    FB:$1,044,310,000,000 (2020 est.)$1,027,240,000,000 (2019 est.)$1.102 trillion (2018 est.)
#    DF:1044310000000000006815744.00

chiRGDP_fBook = getVal( dffBook, 'China',
    'Economy: Real GDP (purchasing power parity)' )
chiRGDP_df = getVal( df, 'China',
    'Economy: Real GDP (purchasing power parity)' )
iraRGDP_fBook = getVal( dffBook, 'Iran',
    'Economy: Real GDP (purchasing power parity)' )
iraRGDP_df = getVal( df, 'Iran',
    'Economy: Real GDP (purchasing power parity)' )

print( f"CHINA REAL GDP\n   FB:{chiRGDP_fBook}\n   DF:{chiRGDP_df}" )
print( f"IRAN REAL GDP\n   FB:{iraRGDP_fBook}\n   DF:{iraRGDP_df:.2f}" )




In [ ]:

listStart = 83
showNumber = 1

In [ ]:
print( f"{listStart}:{listStart + showNumber}" )
for i in list( df.columns )[ listStart:listStart + showNumber ]:
    showTopTen( i, df )
    listStart += showNumber

newDrops = [
    'Government: Country name - etymology',
    'Economy: Real GDP per capita - note',
    'Government: Suffrage'
    ]

In [ ]:
# the Kooky Islands Krew (ctIslands)
#   appears frequently on the bottom/top ten because it is takes less
#   of an event in global terms to have a proportionately large effect
#   on the small sample, be it a population, an area total and so on. 

# Things that China is in the top-ten/5/3/1 of
# (Who else are the "most top 10/5/3/1" countries?)
# combine bottom-tens somehow?

In [ ]:
# Generate CORRELATION DICTIONARY where keys are correlations,
#   values are key-value pairs of baseCol : compareDict
#   CompareDict key-value is col num : correlation coefficient

# correlDict = { }
# baseCol = 1
# while True:
#     colCorrs = { }
#     for colPos in range( baseCol + 1, df.shape[ 1 ] ):
#         colCorrs[ colPos ] = df.iloc[ :, baseCol ].corr( df.iloc[ :, colPos ] )
#     correlDict[ baseCol ] = colCorrs
#     baseCol += 1
#     if baseCol == df.shape[ 1 ]: print( f"Completed correlations" ); break
# 


In [ ]:
# move all most defs to import module, except where useful for process 
# communication
# Heh. mosdef.

# END